
# N3RVU Algebra

This notebook gives a hands‑on tour of the **N3RVU** algebra (baric triple $(r,v,u)$ using the reference implementation in [n3rvu.py](https://github.com/XRReady/NERVU/blob/main/n3rvu.py).  
You'll:

- Build intuition with an **ELI5** explanation.
- Run **quick-start** examples.
- Explore the **ZERO / INV_ZERO** operators and the push–pull between $\mathbb{V}$ (vanished space) and $\mathbb{U}$ (undefined space).
- See a compact **non‑associativity** demo.
- Inspect **right‑division** rules and the **invertibility condition**.
- Try an optional **interactive explorer** (if `ipywidgets` is available).



## Big Picture (from [README](https://github.com/XRReady/NERVU/blob/main/README.md))

> *N3RVU augments the reals with two extra components that **remember** what would normally be lost under multiplying or dividing by zero.*  
We model this with two bookkeeping channels:
- $\mathbb{V}$ (vanished): where things go when you multiply by 0 (“packed into V”).  
- $\mathbb{U}$ (undefined): where things go when you divide by 0 (“packed into U”).

The algebra is **commutative** but **non‑associative** (by design) because the *order* of packing/unpacking between $\mathbb{V}$ and $\mathbb{U}$ matters.



### ELI5 idea
- Multiplying by zero doesn’t **erase**; it **stores**: $a * 0 \to av$.
- Dividing by zero doesn’t **explode**; it **stores** elsewhere: $a / 0 \to au$.
- Doing the inverse operation pulls things **back** (with a push–pull tension between V and U).



### Quick reference (basis and scalar-zero actions):
```text
Basis: e0 (r), e1 (rv), e2 (ru)
a = (a0, a1, a2)

a * 0 = (a2, a0+a1, 0)  # reals move to V; U comes back to real
a / 0 = (a1, 0, a0+a2)  # reals move to U; V comes back to real
```

### What “basis” means

Think of **N3RVU** numbers as 3-slot containers: **(real, V-channel, U-channel)**.  
To build any number, we start with three simple building blocks and add scaled copies:

- $e_0 = (1, -, -)$: “one unit of real stuff”  
- $e_1 = (-, 1, -)$: “one unit stored in V (vanished)”  
- $e_2 = (-, -, 1)$: “one unit stored in U (undefined)”

If a number is written as $a = (a_0, a_1, a_2)$, it just means:

- take $a_0$ copies of $e_0$ (real),  
- and $a_1$ copies of $e_1$ (in V),  
- and $a_2$ copies of $e_2$ (in U).

So `a = (3, 5, −2)` means “3 real, 5 in V, and −2 in U.”



In [ ]:

# --- Setup: import the reference implementation from n3rvu.py ---
from importlib import reload
import math

import sys, os
sys.path.append('/mnt/data')

import n3rvu
reload(n3rvu)
from n3rvu import N3RVU, ZERO, INV_ZERO




## Quick Start

Let's create a few `N3RVU` elements and run the standard operations shown in the README.


In [ ]:

a = N3RVU(1, 2, 3)
b = N3RVU(2, 1, 1)

print("a =", a)
print("b =", b)
print("\n# Bilinear product:")
print("a * b =", a * b)

print("\n# Right-division (solves M(b) x = a):")
try:
    print("a / b =", a / b)
except ZeroDivisionError as e:
    print("a / b raised:", e)

print("\n# Scalar ops:")
print("a / 2 =", a / 2)
print("a * 2 =", a * 2)

print("\n# Special scalar-zero actions:")
print("a * 0 =", a * 0)    # same as a.Z()
print("a / 0 =", a / 0)    # same as a.U()
print("a.Z()  =", a.Z())
print("a.U()  =", a.U())



## Push-Pull Between $\mathbb{V}$ and $\mathbb{U}$

We mirror the example from the [README](https://github.com/XRReady/NERVU/blob/main/README.md) to show how order matters when you combine packing into `V` vs `U`.


In [ ]:

def seq_one(start):
    a = start
    a = a * 0   # pack reals into V
    a = a + N3RVU.e0(2)  # add a real scalar (2)
    a = a / 0   # pack reals into U (while pulling V back to real)
    return a

def seq_two(start):
    a = start
    a = a / 0   # pack reals into U
    a = a + N3RVU.e0(2)
    a = a * 0   # pack reals into V (while pulling U back to real)
    return a

x0 = N3RVU(1,0,0)
print("Start:", x0)
print("(a*0 + 2)/0 =>", seq_one(x0))
print("(a/0 + 2)*0 =>", seq_two(x0))



## Non‑Associativity (compact example)

We replicate the compact multiplication example with basis-like elements:


In [ ]:

x = N3RVU(0,1,0)
y = N3RVU(0,1,0)
z = N3RVU(0,0,1)

xy   = x  * y
yz   = y  * z
xy_z = xy * z
x_yz = x  * yz

print("x     =", x)
print("y     =", y)
print("z     =", z)
print()
print("xy    = x * y   =", xy)
print("yz    = y * z   =", yz)
print()
print("(xy)z = (x*y)*z =", xy_z)
print("x(yz) = x*(y*z) =", x_yz)
print("\nAssociative?" , xy_z == x_yz)



## Right‑Division & Invertibility

Division `a / b` solves the linear system $M(b) \, x = a$, where

$$
\det M(b) \;=\; (b_0^2 - b_1 b_2)\,(b_0 + b_1 + b_2).
$$

So right‑division is **undefined** (raises `ZeroDivisionError`) when either
$$
b_0^2 = b_1 b_2  \quad\text{or}\quad b_0+b_1+b_2 = 0.
$$


In [ ]:

def det_right_map(b: N3RVU) -> float:
    return (b.r*b.r - b.v*b.u) * (b.r + b.v + b.u)

tests = [
    N3RVU(0,0,0),      # trivial singular (not invertible)
    N3RVU(1,1,1),      # b0*b0 = b1b2  (not invertible)
    N3RVU(2,8,0.5),    # b0*b0 = b1b2  (not invertible)
    N3RVU(3,-1,-2),    # b0 + b1 + b2 = 0  (not invertible)
    N3RVU(3,1,1)       # invertible
]

for b in tests:
    print("b =", b, " det(M(b)) =", det_right_map(b))

a = N3RVU(1,2,3)
for b in tests:
    try:
        q = a / b
        print(f"a / {b} =", q)
    except ZeroDivisionError as e:
        print(f"a / {b} -> ZeroDivisionError: {e}")



## (Optional) Interactive Explorer

If `ipywidgets` is available, use the sliders to play with values and see products and special zero actions live.


In [ ]:

try:
    from ipywidgets import interact, FloatSlider, VBox, HBox, HTML
    
    def show_ops(ar, av, au, br, bv, bu):
        a = N3RVU(ar,av,au)
        b = N3RVU(br,bv,bu)
        out = []
        out.append(f"a = {a}   |   b = {b}")
        out.append(f"a*b   = {a*b}")
        try:
            out.append(f"a/b   = {a/b}")
        except ZeroDivisionError as e:
            out.append(f"a/b   = ZeroDivisionError")
        out.append(f"a*0   = {a*0}   |   a/0 = {a/0}")
        out.append(f"weight(a) = {a.weight():.3f}   |   det(M(b)) = {(b.r*b.r - b.v*b.u)*(b.r+b.v+b.u):.3f}")
        return HTML("""<pre style='font-size: 14px;'>""" + "\n".join(out) + """</pre>""")
    
    w = interact(
        show_ops,
        ar=FloatSlider(value=1.0, min=-5, max=5, step=0.1, description='a.r'),
        av=FloatSlider(value=0.0, min=-5, max=5, step=0.1, description='a.v'),
        au=FloatSlider(value=0.0, min=-5, max=5, step=0.1, description='a.u'),
        br=FloatSlider(value=2.0, min=-5, max=5, step=0.1, description='b.r'),
        bv=FloatSlider(value=1.0, min=-5, max=5, step=0.1, description='b.v'),
        bu=FloatSlider(value=1.0, min=-5, max=5, step=0.1, description='b.u'),
    )
except Exception as e:
    print("ipywidgets not available. You can `pip install ipywidgets` locally if desired.")
    print("Error:", e)



---

### Credits & Attribution
This algebra and code were created by **Dale Spencer (Texas Data Safe)**.  
See the project [README](https://github.com/XRReady/NERVU/blob/main/README.md) for the full attribution notice and citation info.
